In [3]:
import random
import os
import re
from glob import glob
from tqdm import tqdm
import numpy as np

import pandas as pd
import ast
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from PIL import Image, ImageDraw, _imaging
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Flatten, Activation, Embedding
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.merge import concatenate
from keras.utils import plot_model
from keras.models import Model
from keras import backend as K
from keras import metrics
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from sklearn.model_selection import train_test_split
import tensorflow as tf
from datetime import datetime
from keras.callbacks import History, ModelCheckpoint
history = History()
from keras.models import model_from_json
import graphviz

Using TensorFlow backend.


In [4]:
def showSampleImgs():
    n_samp = 3
    train_df_sample = train_df.sample(n_samp)
    plt.figure(figsize=(4,4))
    for i in range(n_samp):
        draw = train_df_sample.iloc[i]['drawing']
        label = train_df_sample.iloc[i]['word']
        plt.subplot(n_samp,1,i+1)
        for stroke in draw:
            plt.plot(stroke[0], stroke[1], marker='.', color='black')
            plt.title(label)
            plt.axis('off')
    plt.show()    
    
# Show an image from the dataframe
def showImgFromDf(df, index):
    # Show an image
    plt.imshow(df.iloc[index]['img'],cmap='gray')
    plt.title(df.iloc[index]['word'])
    plt.show()
    
def top_3_accuracy(y_true, y_pred):
    return metrics.top_k_categorical_accuracy(y_true, y_pred, k=3)


# def baseline_conv_model(num_filters, num_classes, img_height, img_width):
#     model = Sequential()
#     model.add(Conv2D(num_filters*1, (5,5), input_shape=(img_height,img_width,1), activation='relu')) 
#     model.add(MaxPooling2D(pool_size=(2,2)))
#     model.add(Dropout(0.2))
#     model.add(Conv2D(num_filters*2, (5,5), activation='relu')) 
#     model.add(MaxPooling2D(pool_size=(2,2)))
#     model.add(Dropout(0.2))
#     model.add(Conv2D(num_filters*4, (3,3), activation='relu')) 
# #     model.add(MaxPooling2D(pool_size=(2,2)))
# #     model.add(Conv2D(num_filters*8, (3,3), activation='relu')) 
#     model.add(Dropout(0.2))
#     model.add(Flatten())
# #     model.add(Dense(2500, activation='relu'))
# #     model.add(Dropout(0.2))
#     model.add(Dense(2000, activation='relu'))
#     model.add(Dropout(0.2))
#     model.add(Dense(1500, activation='relu'))
#     model.add(Dropout(0.2))
#     model.add(Dense(1000, activation='relu'))
#     model.add(Dropout(0.2))
#     # model.add(Dense(num_classes, activation='softmax'))
#     # model.add(Activation('softmax'))
#     model.add(Dense(units=num_classes))
#     model.add(Activation(tf.nn.softmax))

#     # Compile
#     model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', top_3_accuracy, 'categorical_crossentropy'])
#     return model

def baseline_conv_model_functional_API(num_filters, num_classes, img_height, img_width):
    # Input Layer
    inputLayerImg = Input(shape=(img_height,img_width,1))
    
    conv1 = Conv2D(num_filters*1, (5,5), activation='relu')(inputLayerImg)
    pool1 = MaxPooling2D(pool_size=(2,2))(conv1)
    dropout1 = Dropout(0.2)(pool1)
    
    conv2 = Conv2D(num_filters*2, (5,5), activation='relu')(dropout1)
    pool2 = MaxPooling2D(pool_size=(2,2))(conv2)
    dropout2 = Dropout(0.2)(pool2)
    
    conv3 = Conv2D(num_filters*4, (3,3), activation='relu')(dropout2)
    dropout3 = Dropout(0.2)(conv3)
    
    flat = Flatten()(dropout3)
    
    # Add the other scalar inputs as another data stream
    # Continuous variables
    inputLayerScalars = Input(shape=(1,))
        
    # Merge the CNN flat layer and the other input layer
    mergedInput = concatenate([flat, inputLayerScalars])
    
    dense1 = Dense(2000, activation='relu')(mergedInput)
    drop_dense1 = Dropout(0.2)(dense1)
    
    dense2 = Dense(1500, activation='relu')(drop_dense1)
    drop_dense2 = Dropout(0.2)(dense2)
    
    dense3 = Dense(1000, activation='relu')(drop_dense2)
    drop_dense3 = Dropout(0.2)(dense3)
    
    outputLayer = Dense(num_classes, activation=tf.nn.softmax)(drop_dense3)
    
    model = Model(inputs=[inputLayerImg, inputLayerScalars], outputs=outputLayer)
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', top_3_accuracy, 'categorical_crossentropy'])
    return model


In [5]:
random.seed(111)

# Generate class mappings
fnames = glob('train_simplified/*.csv')
tempDf = pd.DataFrame(columns=pd.read_csv(fnames[0], nrows=1).columns)

for name in fnames:
    data = pd.read_csv(name, nrows=2)
    tempDf = tempDf.append(data)
    
ys = tempDf['word']
    
# Convert class labels from categorical to numerical
unique_classes_list = ys.unique()
map_class_to_numeric = {k: v for v, k in enumerate(unique_classes_list)}
map_numeric_to_class = {v: k for k, v in map_class_to_numeric.iteritems()}

countryList = '''['BD','BE','BF','BG','BA','NotAvlbl','BM','BN','BO','JP','BI','BJ','BT','JM','JO','WS','BR','CR','JE','BY','BZ','RU','RW','BB','RS','TL','RE','TM','LR','RO','GU','GT','GR','GP','BH','GY','GG','GF','GE','GD','GB','GA','GN','GM','GL','GI','GH','OM','TN','BU','BW','HR','BS','HT','HU','HK','HN','AD','PR','PS','PW','PT','KN','PY','AI','PA','PF','PG','PE','PK','PH','PL','PM','ZM','EE','EG','ZA','EC','AL','AO','ZZ','ET','ZW','KY','ES','ME','MD','MG','MF','MA','MC','UZ','MM','ML','MO','MN','MH','US','MU','MT','MW','MV','MQ','MP','MS','MR','IM','UG','TZ','MY','MX','IL','FR','SJ','FI','FJ','FM','FO','NI','NL','NO','SO','VU','NC','NF','NG','NZ','NP','CI','CH','CO','CN','CM','CL','CA','CG','CF','OTHER','CD','CZ','CY','CX','UY','CW','CV','SZ','SX','KG','KE','SS','SR','KH','SV','ST','SK','KR','SI','KW','SN','SM','SC','KZ','SA','SG','SE','DO','DM','DJ','DK','VG','DE','YE','DZ','MK','YT','LB','LC','LA','TW','TT','TR','LK','LI','LV','TO','LT','LU','TJ','LS','TH','TG','TC','LY','VC','AE','VE','AG','AF','IQ','VI','IS','IR','AM','IT','VN','AN','AS','AR','AU','AT','AW','IN','AX','AZ','IE','ID','UA','QA','MZ']'''
countryList = eval(countryList)

In [6]:
countryIntMap = {value: int_value+1 for int_value, value in enumerate(countryList)}
# for x,y in enumerate(countryList): 
#     print(y)

In [7]:
# np.array(data['countrycode'].map(countryIntMap)).astype('int32')

In [ ]:
############################### GET NEW FEATURES

In [8]:
# model = baseline_conv_model_functional_API(numStartFilters, numClasses, img_height, img_width)
# from IPython.display import SVG
# from keras.utils.vis_utils import model_to_dot

# SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [124]:
# model.summary()

In [ ]:
############################### GET NEW FEATURES

In [9]:
def draw_it(raw_strokes, output_height, output_width):
#     raw_strokes = eval(raw_strokes)
    image = Image.new("P", (255,255)
#                       , color=1
            )
    image_draw = ImageDraw.Draw(image)

    for stroke in raw_strokes:
        for i in range(len(stroke[0])-1):

            image_draw.line([stroke[0][i], 
                             stroke[1][i],
                             stroke[0][i+1], 
                             stroke[1][i+1]],
                            fill=255, width=6)
    # Reduce image size
    image = image.resize((output_height,output_width),Image.ANTIALIAS)
    
    return np.array(image)


class DataGenerator(keras.utils.Sequence):
    def __init__(self, num_samples_to_train, batch_size, trainGeneratorDataOffsetPerClass, img_width, img_height):
        self.batch_size = batch_size
        self.num_samples_to_train = num_samples_to_train
        self.img_width = img_width
        self.img_height = img_height
        self.trainGeneratorDataOffsetPerClass = trainGeneratorDataOffsetPerClass
    
    def __len__(self):
        return np.ceil(self.num_samples_to_train / float(self.batch_size))
    
    def __getitem__(self, idx):
        
        numRowsEachDf = int(self.batch_size / float(numClasses))
        skipRows = (idx * numRowsEachDf) + self.trainGeneratorDataOffsetPerClass
        img_height = self.img_height 
        img_width = self.img_width
        
        X, y = DataGenerator._getNextData(numRowsEachDf, skipRows, img_height, img_width)
        
        return X, y
    
    @staticmethod
    def _getNextData(numRowsEachDf, skipRows, img_height, img_width):
        
        fnames = glob('train_simplified/*.csv')
        numClasses = len(fnames)

        # Get n rows from all the csv files and append them into one dataframe
        train_df = pd.DataFrame(columns=pd.read_csv(fnames[0], nrows=1).columns)
        
        for name in fnames:
            if skipRows == 0:
                data = pd.read_csv(name, nrows=numRowsEachDf)
            else:
                data = pd.read_csv(name, nrows=numRowsEachDf, skiprows=range(1,skipRows)) # 0th row is the header, so not skipping that
            # Append the training data of all the classes
            train_df = train_df.append(data)
            
        train_df = train_df.reset_index().drop('index', axis=1)
        
        # Some data manipulations
        train_df['countrycode'] = train_df['countrycode'].fillna('NotAvlbl')

        # Shuffle all data points
        train_df = train_df.sample(train_df.shape[0])

        # Get only those which were correctly recognized
        train_df = train_df[train_df['recognized'] == True]

        # Apply eval to drawing object
        train_df['drawing'] = train_df['drawing'].apply(eval)
        
        # Other features from the drawing array
        numStrokes = train_df['drawing'].apply(len)
        
        # Convert drawing to images
        train_df['drawing'] = train_df['drawing'].apply(lambda x: draw_it(x, img_height, img_width))

        # Reshape the array
        train_imgArr = np.vstack(train_df['drawing'].values).flatten().reshape((train_df['drawing'].shape[0], img_height, img_width))
    
        # In Keras, the layers used for two-dimensional convolutions expect pixel values with the dimensions [width][height][pixels] for TF.
        # In the case of RGB, the first dimension pixels would be 3 for the red, green and blue components and it would be like having 3 image inputs for every color image. In this case, the pixel values are gray scale, the pixel dimension is set to 1.
        train_X = train_imgArr.reshape(train_imgArr.shape[0], img_height, img_width, 1).astype('float32')
        
        
         # Initialize the y_train
        y_train = train_df['word']
    
        y_train_numeric = y_train.apply(lambda x: map_class_to_numeric[x])

        # convert class vectors to binary class matrices
        train_y = keras.utils.to_categorical(y_train_numeric, numClasses)
    
        # Get other features
        # Convert numStrokes into numpy array for ingestion [TODO: normalize]
        numStrokes = np.array(numStrokes).astype('float32')
        train_X_scalars = numStrokes
        
        return [train_X, train_X_scalars], train_y
    

In [10]:
img_width = img_height = 32

In [11]:
X_test, y_test = DataGenerator._getNextData(100, 90000, img_height, img_width)

In [12]:
# pp = []
batch_size = 1000
numTrainExamplesPerClass = 500
numClasses = 340
num_samples_to_train = (numTrainExamplesPerClass*numClasses)
trainGeneratorDataOffsetPerClass = 0
dataGenerator = DataGenerator(num_samples_to_train, batch_size, trainGeneratorDataOffsetPerClass, img_width, img_height)

In [13]:
# ############## Remove
# img_width = img_height = 32
# numClasses = 340
# numStartFilters = 8
# model = baseline_conv_model_functional_API(numStartFilters, numClasses, img_height, img_width)
# ############## Remove
# model.summary()

In [15]:
# model = baseline_conv_model(4, 340, 32, 32)
numStartFilters = 8
model = baseline_conv_model_functional_API(numStartFilters, numClasses, img_height, img_width)
model.fit_generator(
        generator=dataGenerator,
        epochs=100,
        verbose=1,
        shuffle=False,
        validation_data=(X_test, y_test)
    
#         ,
#         use_multiprocessing=True,
)
#       steps_per_epoch=(num_training_samples // batch_size),
#       validation_data=my_validation_batch_generator,
#       validation_steps=(num_validation_samples // batch_size),
#       use_multiprocessing=False,
#       workers=16,
#       max_queue_size=32)


Epoch 1/100
170/170 [==============================] - 839s 5s/step - loss: 5.7913 - acc: 0.0112 - top_3_accuracy: 0.0304 - categorical_crossentropy: 5.7913 - val_loss: 5.0135 - val_acc: 0.0440 - val_top_3_accuracy: 0.1069 - val_categorical_crossentropy: 5.0135
Epoch 2/100
170/170 [==============================] - 1184s 7s/step - loss: 4.7468 - acc: 0.0628 - top_3_accuracy: 0.1432 - categorical_crossentropy: 4.7468 - val_loss: 4.3421 - val_acc: 0.1206 - val_top_3_accuracy: 0.2437 - val_categorical_crossentropy: 4.3421
Epoch 3/100
170/170 [==============================] - 1105s 6s/step - loss: 4.2742 - acc: 0.1137 - top_3_accuracy: 0.2318 - categorical_crossentropy: 4.2742 - val_loss: 3.9079 - val_acc: 0.1706 - val_top_3_accuracy: 0.3267 - val_categorical_crossentropy: 3.9079
Epoch 4/100
170/170 [==============================] - 995s 6s/step - loss: 3.9249 - acc: 0.1551 - top_3_accuracy: 0.3038 - categorical_crossentropy: 3.9249 - val_loss: 3.5185 - val_acc: 0.2293 - val_top_3_accura

170/170 [==============================] - 1007s 6s/step - loss: 2.1501 - acc: 0.4362 - top_3_accuracy: 0.6678 - categorical_crossentropy: 2.1501 - val_loss: 2.3756 - val_acc: 0.4264 - val_top_3_accuracy: 0.6383 - val_categorical_crossentropy: 2.3756
Epoch 33/100
170/170 [==============================] - 1159s 7s/step - loss: 2.1328 - acc: 0.4403 - top_3_accuracy: 0.6709 - categorical_crossentropy: 2.1328 - val_loss: 2.3650 - val_acc: 0.4252 - val_top_3_accuracy: 0.6412 - val_categorical_crossentropy: 2.3650
Epoch 34/100
170/170 [==============================] - 1170s 7s/step - loss: 2.1037 - acc: 0.4459 - top_3_accuracy: 0.6760 - categorical_crossentropy: 2.1037 - val_loss: 2.3823 - val_acc: 0.4249 - val_top_3_accuracy: 0.6400 - val_categorical_crossentropy: 2.3823
Epoch 35/100
170/170 [==============================] - 1049s 6s/step - loss: 2.0860 - acc: 0.4478 - top_3_accuracy: 0.6794 - categorical_crossentropy: 2.0860 - val_loss: 2.3987 - val_acc: 0.4268 - val_top_3_accuracy: 0.6

170/170 [==============================] - 1228s 7s/step - loss: 1.7673 - acc: 0.5141 - top_3_accuracy: 0.7459 - categorical_crossentropy: 1.7673 - val_loss: 2.5369 - val_acc: 0.4201 - val_top_3_accuracy: 0.6290 - val_categorical_crossentropy: 2.5369
Epoch 64/100
170/170 [==============================] - 1238s 7s/step - loss: 1.7629 - acc: 0.5134 - top_3_accuracy: 0.7478 - categorical_crossentropy: 1.7629 - val_loss: 2.5448 - val_acc: 0.4187 - val_top_3_accuracy: 0.6289 - val_categorical_crossentropy: 2.5448
Epoch 65/100
170/170 [==============================] - 1356s 8s/step - loss: 1.7457 - acc: 0.5171 - top_3_accuracy: 0.7515 - categorical_crossentropy: 1.7457 - val_loss: 2.5244 - val_acc: 0.4240 - val_top_3_accuracy: 0.6321 - val_categorical_crossentropy: 2.5244
Epoch 66/100
170/170 [==============================] - 1389s 8s/step - loss: 1.7329 - acc: 0.5200 - top_3_accuracy: 0.7534 - categorical_crossentropy: 1.7329 - val_loss: 2.5102 - val_acc: 0.4207 - val_top_3_accuracy: 0.6

170/170 [==============================] - 1276s 8s/step - loss: 1.5753 - acc: 0.5570 - top_3_accuracy: 0.7869 - categorical_crossentropy: 1.5753 - val_loss: 2.6590 - val_acc: 0.4162 - val_top_3_accuracy: 0.6225 - val_categorical_crossentropy: 2.6590
Epoch 95/100
170/170 [==============================] - 1250s 7s/step - loss: 1.5582 - acc: 0.5609 - top_3_accuracy: 0.7894 - categorical_crossentropy: 1.5582 - val_loss: 2.6943 - val_acc: 0.4075 - val_top_3_accuracy: 0.6168 - val_categorical_crossentropy: 2.6943
Epoch 96/100
170/170 [==============================] - 1237s 7s/step - loss: 1.5731 - acc: 0.5581 - top_3_accuracy: 0.7863 - categorical_crossentropy: 1.5731 - val_loss: 2.6611 - val_acc: 0.4138 - val_top_3_accuracy: 0.6192 - val_categorical_crossentropy: 2.6611
Epoch 97/100
170/170 [==============================] - 3391s 20s/step - loss: 1.5802 - acc: 0.5555 - top_3_accuracy: 0.7856 - categorical_crossentropy: 1.5802 - val_loss: 2.7003 - val_acc: 0.4058 - val_top_3_accuracy: 0.

In [10]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))
# model

CNN Error: 29.11%


In [17]:
# model = 0

In [132]:
# %%time
# # Transform data for CNN
# X_train, X_test, y_train, y_test, imgArr_test, map_class_to_numeric, map_numeric_to_class = CNN_dataPrep(train_df, img_height, img_width)

In [2]:
# num_filters = 2
# num_classes = 340
# continueTrain = False
# if continueTrain == False:
#     model = baseline_conv_model(num_filters, num_classes, img_height, img_width)

# # checkpoint
# filepath="Saved_Models/weights.best.hdf5"
# checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
# callbacks = [checkpoint]

# print(model.summary())

# hist = model.fit(X_train, 
#           y_train, 
#           validation_data=(X_test, y_test),
#           epochs=2, 
#           batch_size=5, 
#           verbose=2,
#           callbacks = callbacks)

In [37]:
# Save the model
# serialize model to JSON
model_json = model.to_json()
with open(("Saved_Models/model_" + str(datetime.today())[:16].replace(" ", "_").replace(":","-") + ".json"), "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(("Saved_Models/model_" + str(datetime.today())[:16].replace(" ", "_").replace(":","-") + ".h5"))
print("Saved model to disk")

Saved model to disk


In [25]:
# model = ""

In [24]:
# Load the model back
# load json and create model
json_file = open('Saved_Models/model_2018-11-21_02-44.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("Saved_Models/model_2018-11-21_02-44.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', top_3_accuracy, 'categorical_crossentropy'])
# score = loaded_model.evaluate(X, Y, verbose=0)
# print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))
model = loaded_model

Loaded model from disk


# Create Test Data Set

In [38]:
def getTestData(img_height, img_width):
        
        numClasses = 340

        test_df = pd.read_csv('test_simplified.csv')            

        # Convert drawing to images
        test_df['drawing'] = test_df['drawing'].apply(lambda x: draw_it(x, img_height, img_width))

        # Reshape the array
        test_imgArr = np.vstack(test_df['drawing'].values).flatten().reshape((test_df['drawing'].shape[0], img_height, img_width))
    
        # In Keras, the layers used for two-dimensional convolutions expect pixel values with the dimensions [width][height][pixels] for TF.
        # In the case of RGB, the first dimension pixels would be 3 for the red, green and blue components and it would be like having 3 image inputs for every color image. In this case, the pixel values are gray scale, the pixel dimension is set to 1.
        test_X = test_imgArr.reshape(test_imgArr.shape[0], img_height, img_width, 1).astype('float32')
                
        return test_df, test_X

In [39]:
test_df, test_X = getTestData(img_width, img_height)

In [40]:
# Make predictions
preds = model.predict(test_X)
outputDf = test_df.copy()[['key_id']]
outputDf['word'] = ''
map_numeric_to_class_space_normal = map_numeric_to_class.copy()
for key in map_numeric_to_class_space_normal:
    map_numeric_to_class_space_normal[key] = (map_numeric_to_class_space_normal[key].replace(" ", "_"))

for i in tqdm(range(preds.shape[0])):
    outputDf['word'].at[i] = ' '.join(([map_numeric_to_class_space_normal[predClass] for predClass in [tup[1] for tup in sorted(zip(preds[i], range(340)), reverse=True)[:3]]]))

# Create csv
outputDf.to_csv('initial_pred.csv', index=False)

100%|██████████| 112199/112199 [00:22<00:00, 5018.47it/s]
